In [33]:
import hashlib
def format_tel(tel):
    """
    电话号码，格式化处理（只保留数字）
    """
    if isinstance(tel, (str)):
        return ''.join(i for i in tel if i.isdigit())
    else:
        return None

def phone_encode(string, method="md5_sha1"):
    """
    先MD5，再sha1加密
    """
    try:
        if 'md5' in method:
            m = hashlib.md5()
            m.update(string.encode(encoding='utf-8'))
            string = m.hexdigest()
            string = string[0:32]
        if 'sha1' in method:
            s = hashlib.sha1()
            s.update(string.encode(encoding='utf-8').upper())
            string = s.hexdigest().upper()
        return string
    except Exception as e:
        return None

In [7]:
import pandas as pd
cuishou = pd.read_csv('cuishou_db', header=None,names=['tel', 'type'])
cy = pd.read_csv('canyin.tel', header=None,names=['tel'])
cy['tel'] = cy.tel.map(format_tel).map(phone_encode)
cy.iloc[:]

FileNotFoundError: File b'cuishou_db' does not exist

In [35]:
mapping = {
    "cy": {
#         1: (0, 187432),
#         4: (187432,220703+187432)
        1: (0, 7-1),
        4: (7, 7+5-1)
    },
    "cs": {
        2: (0,48732-1),
        5: (48732, 48732+22389-1),
        7: (48732+22389, 48732+22389+72084-1),
        8: (48732+22389+72084, -1)
    },
    "ys": {
        3: (0,193302),
        6: (193302,-1)
    }
}

In [36]:
cs = cuishou[cuishou.type==1].reset_index(drop=True)
ys = cuishou[cuishou.type==2].reset_index(drop=True)

In [22]:
cs.loc[0:6]

,tel,type
0,DD39B0F32D1AE17B64575AF6FD1F57C4D2957A97,1
1,41F798A8BE2CBFEFB8F1E7D71656ADDAA5A4226E,1
2,EBA98CD0C282F048A12D49865E2B17FB79BDC410,1
3,2A5FCC03F356A895B2C06962DF5F3B659710A1CB,1
4,BC6264931AFD2416092734D6A84462DAD6CD4A02,1
5,40272DCC69040A84D63EDB8BCF1D81B72F4A211E,1
6,3FC82EA3C4009DFA0985EAA315C7E942F8A42217,1


In [37]:
dic = {}
result = pd.DataFrame()
for types,item in mapping.items():
    for k,v in item.items():
        print(types,k,v)
        if v[1] != -1:
            eval(types).loc[v[0]:v[1], 'type'] = k
        else:
            eval(types).loc[v[0]:, 'type'] = k
    if types == 'cy':
        cy = cy.dropna()

cy 1 (0, 6)
cy 4 (7, 11)
cs 2 (0, 48731)
cs 5 (48732, 71120)
cs 7 (71121, 143204)
cs 8 (143205, -1)
ys 3 (0, 193302)
ys 6 (193302, -1)


In [38]:
a = ys.append(cs).append(cy).sort_values('type')
a['type'] = a.type.astype(int)
a.to_csv('new_cuishou_db',index=False,header=False)
a.groupby('type').count()

,tel
type,
1,7
2,48732
3,193302
4,5
5,22389
6,428558
7,72084
8,14224


In [13]:
with open('new_cuishou_db','r') as rp:
    data = rp.read()
print(data[:1000])

4C8BD6C0D70253160252F419A562B6BF2211780B,1
47F3BCEB0E5DCB021ACD93080CC2C725A7E49B15,1
5F59530B578EA5A76918FAB71E46511721DE8D2D,1
ACF5B4A7050ABD8E4C6E281090E56C84BC07F234,1
F0133613390BCA438870F44CD98E14540A82DEEE,1
277E95050160E8917A4C2E8542E19ED56978B8EA,1
B42EBEFD01D0C98130CFAD5D54E0B7C777B119BA,1
A6DF058861EFEBE0A615EBEF9EA9D2EDDC6F17A5,2
70692E514CD7424C7D2CFA526333D081FF01DE03,2
AB9B128924E071AFA56FB2245014BA18EDCA4192,2
F80633AB664A7099387FFC52C2E4F96F1B482ABB,2
769087904730040730A28C034DE8D9950417861B,2
7BCF01815ABB44B6F6B04F08939E6411D4408D4E,2
4808B6C3232A48C079657751B2BC85BB4067E742,2
CB8FD2EA2C8690AFCD5E2C0DEFA2CC5539DAAA5D,2
041CDB6FFFCE8611B55468B8A1BA03FFD8E41DF4,2
F0D9AEFB862A3960889F05DA3B5B80C1F1DA3199,2
6AA87DC69D80333C24EE508D6335E547BACDC0EA,2
C51898EA7F8DC8DEA3D0F63D7AF3E388C4635498,2
02AD66FE613CDB3A5465E98517CF774C5F275603,2
84CB9375666775FF83144DBE876B657EFDDB8BC3,2
21BB11B05568EBC8D2BA79F8C1485191C6D04004,2
4C8F8FA62D811C4BD5F9DC50973CB2C739DD6239,2
8F1A13AEDFF

In [29]:

b = pd.read_csv('new_cuishou_db', header=None, names=['tel', 'type'])
b.type.unique()

array([1, 2, 3, 4, 5, 6, 7, 8])

In [39]:
a = pd.DataFrame()
a.groupby('type').count()

KeyError: 'type'

In [6]:
'01080808080'[:3]

'010'

In [10]:
cy = pd.read_csv('canyin.tel', header=None,names=['tel'])

In [13]:
def format_tel(tel):
    if isinstance(tel, str):
        tmp = ''.join(i for i in tel if i.isdigit())
        if tmp[:2] in ['01', '02']:
            return (tmp[:3], tmp[3:])
        elif tmp.startswith('0'):
            return (tmp[:4], tmp[4:])
        else:
            return (0, tmp)
    else:
        return None


In [17]:
type(cy.tel.map(format_tel)[0])

tuple

In [29]:
cy.tel.map(format_tel).apply(pd.Series).add_prefix('val_')

,val_0,val_1
0,0813,7986935
1,0,18144231750
2,0813,5150299
3,0,13890010129
4,0,15284818689
5,0,13568332513
6,0,13698253766
7,0,15984153461
8,0,15281366075
9,0,13890053586


In [28]:
cy.assign(**cy.tel.map(format_tel).apply(pd.Series).add_prefix('val_'))

,tel,val_0,val_1
0,0813-7986935,0813,7986935
1,18144231750,0,18144231750
2,0813-5150299,0813,5150299
3,13890010129,0,13890010129
4,15284818689,0,15284818689
5,13568332513,0,13568332513
6,13698253766,0,13698253766
7,15984153461,0,15984153461
8,15281366075,0,15281366075
9,13890053586,0,13890053586


In [26]:
(cy.tel.map(format_tel).apply(pd.Series)).rename(columns={0:'tel',1:'code'})

,tel,code
0,0813,7986935
1,0,18144231750
2,0813,5150299
3,0,13890010129
4,0,15284818689
5,0,13568332513
6,0,13698253766
7,0,15984153461
8,0,15281366075
9,0,13890053586
